We start by importing the OpenFst Python wrapper:

In [ ]:
import openfst_python as fst

Then we create tables for our symbols

In [ ]:
input_sym = fst.SymbolTable()
output_sym = fst.SymbolTable()

input_sym.add_symbol('<eps>') # by convention, <eps> always
                              # has symbol zero
input_sym.add_symbol('a')  # input symbols
input_sym.add_symbol('b')

output_sym.add_symbol('<eps>')  # output symbols
output_sym.add_symbol('d')
output_sym.add_symbol('c')

A **SymbolTable()** is simply a table associating symbols and indexes. We add symbols to the table with the method **add_symbol()**

Now that we've got our symbol tables, we will build the FST itself:

In [ ]:
f = fst.Fst()

f.set_input_symbols(input_sym)
f.set_output_symbols(output_sym)

Our FST transduces the input to the outputs, so we set the symbol tables as such. Now, we want to add a number of states:

In [ ]:
s0 = f.add_state()
s1 = f.add_state()
s2 = f.add_state()
s3 = f.add_state()

The output of the **add_state()** method is just the index assigned to the state, but it can be useful to assign that index to a variable to give it a more meaningful label.

To create arcs between states, we do:

In [ ]:
a = input_sym.find('a')
b = input_sym.find('b')
c = output_sym.find('c')
d = output_sym.find('d')

f.add_arc(s0, fst.Arc(a, c, None, s1))
f.add_arc(s0, fst.Arc(b, d, None, s2))
f.add_arc(s1, fst.Arc(a, c, None, s3))
f.add_arc(s2, fst.Arc(b, d, None, s3))

The syntax for the method **add_arc** is:

**add_arc(*source state*, *arc to add*)**

while the syntax for initializing a class **Arc()** is:

**Arc(*input symbol index*, *output symbol index*, *weight*, *destination state*)**

We use the **find()** method of the symbol tables to get the index of a certain label.

Now we just add start and end states:

In [ ]:
f.set_start(s0)
f.set_final(s3)

And voila, our first FST, shown in the example above, is done!

# Exercises

In [ ]:
def parse_lexicon(lex_file):
    """
    Parse the lexicon file and return it in dictionary form.
    
    Args:
        lex_file (str): filename of lexicon file with structure '<word> <phone1> <phone2>...'
                        eg. peppers p eh p er z

    Returns:
        lex (dict): dictionary mapping words to list of phones
    """
    
    lex = {}  # create a dictionary for the lexicon entries (this could be a problem with larger lexica)
    with open(lex_file, 'r') as f:
        for line in f:
            line = line.split()  # split at each space
            lex[line[0]] = line[1:]  # first field the word, the rest is the phones
    return lex

In [ ]:
lex = parse_lexicon('lexicon.txt')
lex

1. Write a function that generates symbol tables from a lexicon, using the provided phonetic dictionary in **lexicon.txt** and the helper function *parse_lexicon* in cells above. Use p_1, p_2, ..., eh_1, ... to denote the phone states. 

In [ ]:
def generate_symbol_tables(lexicon, n=3):
    '''
    Return word, phone and state symbol tables based on the supplied lexicon
    
    Args:
        lexicon (dict): lexicon to use, created from the parse_lexicon() function
        n (int): number of states for each phone HMM
        
    Returns:
        word_table (fst.SymbolTable): table of words
        phone_table (fst.SymbolTable): table of phones
        state_table (fst.SymbolTable): table of HMM phone-state IDs
    '''
        
    state_table = fst.SymbolTable()
    phone_table = fst.SymbolTable()
    word_table = fst.SymbolTable()

    state_table.add_symbol('𝜖')
    phone_table.add_symbol('𝜖')
    word_table.add_symbol('𝜖')
    phones = set()
    
    for word, word_phones in sorted(lexicon.items(), key=lambda x: x[0]):
        word_table.add_symbol(word)
        print(word)
        print(word_phones)
        phones = phones.union({p for p in word_phones})

    for phone in sorted(phones):
        phone_table.add_symbol(phone)
        for i in range(1, n + 1):
            print(f"{phone}_{i}")
            state_table.add_symbol(f"{phone}_{i}")

    return word_table, phone_table, state_table

In [ ]:
word_table, phone_table, state_table = generate_symbol_tables(lex)

2. Create functions generating WFSTs representing the following HMMs:

    1. A 3-state left-to-right phone HMM with self-loops
    2. A parallel-path left-to-right model (shown on Slide 10 of Lecture 4)
    3. An ergodic HMM with $n$ states (you will need to think about how to handle final states)

In [ ]:
def generate_phone_wfst(f, start_state, phone, n):
    """
    Generate a WFST representing an n-state left-to-right phone HMM.
    
    Args:
        f (fst.Fst()): an FST object, assumed to exist already
        start_state (int): the index of the first state, assumed to exist already
        phone (str): the phone label 
        n (int): number of states of the HMM excluding start and end
        
    Returns:
        the final state of the FST
    """
    
    current_state = start_state
    
    for i in range(1, n+1):

        in_label = state_table.find('{}_{}'.format(phone, i))
        out_label = state_table.find('𝜖')
        
        # special case for last state
        if i == n:
            f.add_arc(current_state, fst.Arc(in_label, out_label, None, current_state))
            continue
        
        next_state = f.add_state()
        
        f.add_arc(current_state,fst.Arc(in_label, out_label, None, current_state))
        f.add_arc(current_state,fst.Arc(in_label, out_label, None, next_state))
        current_state = next_state
        
    final_state = f.add_state()
    f.add_arc(current_state, fst.Arc(in_label, phone_table.find(phone), None, final_state))
    current_state = final_state

    return current_state

f = fst.Fst()
start = f.add_state()
f.set_start(start)

last_state = generate_phone_wfst(f, start, 'p', 3)

f.set_input_symbols(state_table)
f.set_output_symbols(phone_table)

In [ ]:
def generate_parallel_path_wfst(f, start_state, n):
    """
    Generate a WFST representing an n-state parallel-path left-to-right HMM
    
    Args:
        f (fst.Fst()): an FST object, assumed to exist already
        start_state (int): the index of the first state, assumed to exist already
        n (int): number of states of the HMM excluding start and end
        
    Returns:
        the final state of the FST
    """
    
    current_state = start_state
    next_state = start_state
    
    for i in range(n):
    
        in_label = state_table.find('𝜖')
        out_label = state_table.find('𝜖')

        if i < n - 1:
            if i == 0:
                next_state = f.add_state()
                f.add_arc(current_state, fst.Arc(in_label, out_label, None, current_state))
                f.add_arc(current_state, fst.Arc(in_label, out_label, None, next_state))
                f.add_arc(next_state, fst.Arc(in_label, out_label, None, next_state))
        
            next_next_state = f.add_state()
            f.add_arc(current_state, fst.Arc(in_label, out_label, None, next_next_state))
            f.add_arc(next_state, fst.Arc(in_label, out_label, None, next_next_state))
            
            if i < n - 2:
                f.add_arc(next_next_state, fst.Arc(in_label, out_label, None, next_next_state))
        
        current_state = next_state
        next_state = next_next_state

    f.set_final(current_state)
    return current_state

f = fst.Fst()
start = f.add_state()
f.set_start(start)

last_state = generate_parallel_path_wfst(f, start, 4)

f.set_input_symbols(state_table)
f.set_output_symbols(phone_table)

In [ ]:
def generate_ergodic_wfst(f, start_state, n):
    """
    Generate a WFST representing an n-state ergodic HMM
    
    Args:
        f (fst.Fst()): an FST object, assumed to exist already
        start_state (int): the index of the first state, assumed to exist already
        n (int): number of states of the HMM excluding start and end
        
    Returns:
        the final state of the FST
    """    
    
    current_state = start_state
    
    for i in range(n+1):
        
        in_label = state_table.find('𝜖')
        out_label = state_table.find('𝜖')
        
        if i == 0:
            for _ in range(1, n+1):
                f.add_arc(current_state, fst.Arc(in_label, out_label, None, f.add_state()))
                
        else:
            for j in range(1, n+1):
                f.add_arc(current_state, fst.Arc(in_label, out_label, None, j))

        current_state = i+1

    final_state = f.add_state()
    f.set_final(final_state)
    for i in range(1, n+1):
        f.add_arc(i, fst.Arc(in_label, out_label, None, final_state))
    current_state = final_state
    return current_state

f = fst.Fst()
start = f.add_state()
f.set_start(start)

last_state = generate_ergodic_wfst(f, start, 5)

f.set_input_symbols(state_table)
f.set_output_symbols(phone_table)

3. Write a function to generate an HMM for any word in the lexicon using $n$ states per phone by concatenating $n$-state phone HMMs. Make use of your *generate_phone_wfst()* function. 

In [ ]:
def generate_word_wfst(f, start_state, word, n):
    """ Generate a WFST for any word in the lexicon, composed of n-state phone WFSTs.
        This will currently output word labels.  
        Exercise: could you modify this function and the one above to output a single 
        phone label instead?
    
    Args:
        f (fst.Fst()): an FST object, assumed to exist already
        start_state (int): the index of the first state, assumed to exist already
        word (str): the word to generate
        n (int): states per phone HMM
        
    Returns:
        the constructed WFST
    
    """

    current_state = start_state
    
    for phone in lex[word]:
        last_state = generate_phone_wfst(f, current_state, phone, n)
        current_state = last_state
        
#     f.set_final(current_state)
    
    return f

f = fst.Fst()
start = f.add_state()
f.set_start(start)

generate_word_wfst(f, start, 'peppers', 3)
f.set_input_symbols(state_table)
f.set_output_symbols(phone_table)

from subprocess import check_call
f.draw('tmp.dot', portrait=True)
check_call(['dot', '-Tpng', '-Gdpi=200', 'tmp.dot', '-o', 'tmp.png'])
from IPython.display import Image
Image(filename='tmp.png')

4. Generate an HMM that can recognise:

    1. any phone contained in the lexicon
    2. any sequence of phones contained in the lexicon
    
Think of the difference between the first and the second HMM. Again, previous functions should help in the task. 

In [ ]:
def generate_phone_recognition_wfst(n):
    """ generate a HMM to recognise any single phone in the lexicon
    
    Args:
        n (int): states per phone HMM

    Returns:
        the constructed WFST
    
    """
    
    f = fst.Fst()
    
    # create a single start state
    start_state = f.add_state()
    f.set_start(start_state)
    # create a single end state
    end_state = f.add_state()
    f.set_final(end_state)
    
    # collect all unique phones from the lexicon
    phones = sorted({phone for word, phones in lex.items() for phone in phones})

    for phone in phones:
        phone_start_state = f.add_state()
        phone_end_state = generate_phone_wfst(f, phone_start_state, phone, n)
        # transition into phone HMM from global start state
        f.add_arc(start_state, fst.Arc(state_table.find('𝜖'), state_table.find('𝜖'), None, phone_start_state))
        # transition from phone HMM into global end state
        f.add_arc(phone_end_state, fst.Arc(state_table.find('𝜖'), state_table.find('𝜖'), None, end_state))

    return f

f = generate_phone_recognition_wfst(3)
f.set_input_symbols(state_table)
f.set_output_symbols(phone_table)

from subprocess import check_call
f.draw('tmp.dot', portrait=True)
check_call(['dot', '-Tpng', '-Gdpi=300', 'tmp.dot', '-o', 'tmp.png'])
from IPython.display import Image
Image(filename='tmp.png')

In [ ]:
def generate_phone_sequence_recognition_wfst(n):
    """ generate a HMM to recognise any single phone sequence in the lexicon
    
    Args:
        n (int): states per phone HMM

    Returns:
        the constructed WFST
    
    """
    
    f = fst.Fst()
    
    # create a single start state
    start_state = f.add_state()
    f.set_start(start_state)
    # create a single end state
    end_state = f.add_state()
    f.set_final(end_state)
    
    # collect all unique phones from the lexicon
    phones = sorted({phone for word, phones in lex.items() for phone in phones})

    for phone in phones:
        phone_start_state = f.add_state()
        phone_end_state = generate_phone_wfst(f, phone_start_state, phone, n)
        # transition into phone HMM from global start state
        f.add_arc(start_state, fst.Arc(state_table.find('𝜖'), state_table.find('𝜖'), None, phone_start_state))
        # transition from phone HMM to gloabl end state
        f.add_arc(phone_end_state, fst.Arc(state_table.find('𝜖'), state_table.find('𝜖'), None, end_state))
    
    # loop back to start state to recognize more phones
    f.add_arc(end_state, fst.Arc(state_table.find('𝜖'), state_table.find('𝜖'), None, start_state))

    return f

f = generate_phone_sequence_recognition_wfst(3)
f.set_input_symbols(state_table)
f.set_output_symbols(phone_table)

from subprocess import check_call
f.draw('tmp.dot', portrait=True)
check_call(['dot', '-Tpng', '-Gdpi=300', 'tmp.dot', '-o', 'tmp.png'])
from IPython.display import Image
Image(filename='tmp.png')

5. You have decided that the best way to start teaching a friend English is to have them learn the notorious tongue-twister *peter piper picked a peck of pickled peppers*.  Write a function that creates an HMM that can recognize any sequence of these words.

In [ ]:
def generate_word_sequence_recognition_wfst(n):
    """ generate a HMM to recognise any single word sequence for words in the lexicon
    
    Args:
        n (int): states per phone HMM

    Returns:
        the constructed WFST
    
    """
    
    f = fst.Fst()
    
    # create a single start state
    start_state = f.add_state()
    f.set_start(start_state)
    # create a single end state
    end_state = f.add_state()
    f.set_final(end_state)
    
    for word in lex.keys():
        current_state = f.add_state()
        f.add_arc(start_state, fst.Arc(state_table.find('𝜖'), state_table.find('𝜖'), None, current_state))
    
        for phone in lex[word]:
            current_state = generate_phone_wfst(f, current_state, phone, n)
        
        # transition from word HMM into global end state
        f.add_arc(current_state, fst.Arc(state_table.find('𝜖'), state_table.find('𝜖'), None, end_state))
        
    # loop back to start state to recognize more words
    f.add_arc(end_state, fst.Arc(state_table.find('𝜖'), state_table.find('𝜖'), None, start_state))
        
    return f

f = generate_word_sequence_recognition_wfst(3)
f.set_input_symbols(state_table)
f.set_output_symbols(phone_table)

from subprocess import check_call
f.draw('tmp.dot', portrait=True)
check_call(['dot', '-Tpng', '-Gdpi=300', 'tmp.dot', '-o', 'tmp.png'])
from IPython.display import Image
Image(filename='tmp.png')

# If you have more time

Probabilities in WFSTs are traditionally expressed in negative log format, that is, the weight $w$ on an arc transitioning between states $i$ and $j$ is given by $w=-\log a_{ij}$, where $a_{ij}$ is the HMM transition probability.

6. Add weights to your WFSTs corresponding to transition probabilities.  Assume that the probability of a self-loop is $0.1$, and that when transitioning *between* separate phones or words, the probabilities are uniform over all transitions.